## Modelo de transito de crecidas

## 1. Introducción

Cuando se presenta un evento de lluvia, sobre el cauce principal de la cuenca se genera una creciente de respuesta. Las crecientes se reflejan en hidrógrafas, las cuales se componen de ascenso, pico y recesión. El efecto de la creciente sobre el cauce es una traslación aguas abajo, conocido como transito de crecidas.

La velocidad a la que se traslada la crecida depende de la pendiente del cuace, la cobertura del lecho, la magnitud del evento y en general de los párametros determinantes de la resistencia al flujo. 

El transito puede ser entendido como el movimiento longitudinal de la onda de crecida. El paso de la creciente entre dos puntos sobre el cauce genera un diferencial de tiempo que puede ser aprovechado para pronosticar niveles en el corto plazo.

Considerando que el SIATA cuenta con estaciones que se encuentran ubicadas sobre el mismo cauce se desarrola un modelo estadístico para determinar en tiempo real el prónostico de nivel máximo en las estaciones aguas abajo de acuerdo a los eventos historicos. Inicialmente se exploran las estaciones de nivel ubicadas a lo largo del Río Medellín, de sur a norte (sentido del flujo), 106. Tres Aguas, 179. La Inmaculada, 91. Estación Sabaneta, 94. Aguacatala, 93. Puente La 33, 99. Aula Ambiental y 140. Puente Fundadores.

El nivel maximo probable en una estación se calcula dado el máximo nivel de la estación aguas arriba mas cercana, los parámetros que se pueden estimar son nivel máximo y tiempo en que ocurriría (tiempo de viaje).

En este ejemplo se presentan las hidrografas de crecida en dos estaciones ubicadas sobre el Río Medellín 94. Aguacatala (aguas arriba) y 93. Puente La 33 (aguas abajo).

![title](nivel_transito.png)

## 2. Esquema del modelo

En Modelo de Transito se basa en reglas de pronostico elaboradas a partir de registros historicos de crecientes. Las relaciones entre nivel de entrada y nivel de salida se realiza por pares entre estaciones consecutivas. 

Las variables pronosticadas son nivel aguas abajo y tiempo de viaje (tiempo que debe trascurrir para que se presente la creciente aguas abajo). Ambas variables estan asociadas a una incertidumbre.

![title](esquema_transito.png)

## 3. Información utilizada

Para la ejecución del Modelo de Transito se requiere información de niveles máximos de los eventos identificados en la historia. Esta información es almacenada en una base de datos y utilizada en el modelo operacional como regla de pronostico.

La información utilizada en el modelo es registrada por el Sistema de Alerta Temprana de Medellín y el Valle de Aburrá (SIATA) a través de las red de nivel, la cual opera en tiempo real con resolución temporal de $1min$.

### 3.1 Niveles

En las estaciones donde se aplica el modelo de transito se cuenta con un sensor de nivel. De la serie historica se extraen los niveles máximos (nivel pico de las hidrografas de respuesta) para componer la base de datos de eventos. 

Los niveles máximos son hallados con una metodología de doble gradiente que se aplica sobre la serie total. Debido a la resolución temporal tan fina es necesario identificar los máximos con la serie suavizada para eliminar el ruido de la serie original.

A continuación se presentan algunas gráficas que resumen el comportamiento de los máximos en cada una de las cuencas.

Histograma de niveles máximos en la estación Aula Ambiental sobre el Río Medellín. Gran parte de los eventos representan respuestas de magnitud baja, el objetivo de los pronosticos de crecida se concentra en los eventos de magnitud media y alta, que aunque ocurren con menor frecuencia son los que generan afectaciones importantes.

![title](hist_niveles_aula.png)

Ciclo horario de niveles máximos en la estación Aula Ambiental sobre el Río Medellín. Los eventos de magnitud alta ocurren con mayor frecuencia en la madrugada ( 1am  -  4am ) y en la tarde ( 3pm  -  6pm ), siendo mucho mas frecuentes los de la tarde. Este comportamiento esta ligado a la formación de sistemas de lluvia convectivos que en el Valle de Aburrá generan lluvia de alta intensidad.

![title](ciclo_horario_aula.png)


## 4. Estaciones del modelo

El Modelo de Transito esta operativo para $6$ estaciones ubicadas sobre el Río Medellín. 

Las tramos entre estaciones son de distintos coberturas, pendientes, alineación en planta y longitudes. En este sentido se piensan desarrollar relaciones posteriores que complementen la base física del Modelo de Transito. Las estaciones consideradas hasta ahora son las siguientes:

In [15]:
import numpy as np

nivel = [106,179,91,94,93,99,140]
cuenca = ['Río Medellín - Tres Aguas','Río Medellín - La Inmaculada','Río Medellín - Metro Sabaneta',
          'Río Medellín - Aguacatala','Río Medellín - Puente La 33','Río Medellín - Aula Ambiental',
          'Río Medellín - Puente Fundadores']
municipios = ['Caldas','Caldas','Sabaneta','Medellín','Medellín','Medellín','Copacabana']
areas = [96.48,123.83,142.86,296.80,356.57,391.95,737.44]
pendiente_media = [48.65,40.29,39.44,37.07,35.61,34.60,32.61]

import pandas as pd
df = pd.DataFrame(index=nivel,columns = ['Cuenca','Municipio','Area','Pendiente media'])

df.loc[nivel,'Area'] = areas
df.loc[nivel,'Cuenca'] = cuenca
df.loc[nivel,'Municipio'] = municipios
df.loc[nivel,'Pendiente media'] = pendiente_media

df.sort_values('Area',ascending=True)

,Cuenca,Municipio,Area,Pendiente media
106,Río Medellín - Tres Aguas,Caldas,96.48,48.65
179,Río Medellín - La Inmaculada,Caldas,123.83,40.29
91,Río Medellín - Metro Sabaneta,Sabaneta,142.86,39.44
94,Río Medellín - Aguacatala,Medellín,296.8,37.07
93,Río Medellín - Puente La 33,Medellín,356.57,35.61
99,Río Medellín - Aula Ambiental,Medellín,391.95,34.6
140,Río Medellín - Puente Fundadores,Copacabana,737.44,32.61


La idea es lograr la mayor cobertura de cauces posibles, en el momento se tienen algunos afluentes con dos estaciones de nivel, se requiere aumentar la historia de eventos para que sea representativo.

En el siguiente mapa se muestra la cobertura actual de las estaciones consideradas y su distribución a lo largo del Río Medellín.

![title](mapa_transito.png)

## 5. Modelo operacional

El Modelo de Transito se ejecuta operacionalmente cada $2$ minutos para garantizar entrada de registros actualizados de nivel. El resultado para pronostico de nivel es presentado en graficas y relacionado con los niveles de alerta medidos en campo.

Los niveles de alerta son referencias tomadas en campo para categorizar el riesgo ante el aumento de niveles en el cauce. Con los niveles de alerta se identifican estructuras que se ven comprometidas, viviendas, vias, entregas de alcantarillado, etc.
Las categorias de alerta son: nivel de atención, nivel alerte moderado, compromiso de estructuras y compromiso de vidas humanas.

Este es un ejemplo para la estación Aula Ambiental en donde se observa la evolución del nivel registrado por el sensor respecto al nivel y tiempo pronosticado con el Modelo de Transito. Las regiones sombreadas representan incertidumbre para los percentiles 10,25,75 y 90, el punto negro indica el percentil 50. La incertidumbre se presenta tanto para el nivel máximo como para el tiempo de ocurrencia.

![title](animacion.gif)

In [14]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### 5.1 Actualización de eventos

El Modelo de Transito operacional se ejecuta tomando como base las reglas de pronostico generadas con los eventos historicos. Dichas reglas deben ser actualizadas con los eventos nuevos para aumentar la historia y por consiguiente mejorar la representativa estadistica.

Para actualizar de manera automatica los eventos del Modelo de Transito se utiliza el script $actualizar2$.